# Drifter Simulation in the JDF

In [1]:
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime, timedelta
from dateutil.parser import parse
from scipy.io import loadmat

from parcels import FieldSet, ParticleSet, JITParticle, ErrorCode, AdvectionRK4

%matplotlib inline 
# ^ a magic funtion that renders the figure in the notebook right after the cell and will be stored in the notebook document

INFO: Compiled ParcelsRandom ==> /var/folders/r6/_k68dfg51w9dl1vklfcg7xcm0000gn/T/parcels-501/libparcels_random_7cc805c7-461d-4e32-be91-da5c9fe2dd3b.so


### Local Functions

In [2]:
def make_prefix(date, path, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., ./SalishSea/SalishSea_1h_yyyymmdd_yyyymmdd
    """

    datestr = '_'.join(np.repeat(date.strftime('%Y%m%d'), 2))
    prefix = os.path.join(path, f'SalishSea_1{res}_{datestr}')
    
    return prefix


def DeleteParticle(particle, fieldset, time):
    """Delete particle from OceanParcels simulation to avoid run failure
    """
    
    print(f'Particle {particle.id} lost !! [{particle.lon}, {particle.lat}, {particle.depth}, {particle.time}]')
    particle.delete()

### Load Drifters and Definitions

In [3]:
# Define paths
paths = {
    'NEMO': './SalishSea/',
    'coords': './grid/coordinates_seagrid_SalishSea201702.nc',
    'mask': './grid/mesh_mask201702.nc',
    'out': './results',
}

In [4]:
coords = xr.open_dataset(paths['coords'], decode_times=False)
mask = xr.open_dataset(paths['mask'])

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/rebeccabeutel/ocean/analysis-becca/OceanParcels/grid/coordinates_seagrid_SalishSea201702.nc'

In [5]:
# Duration and timestep [s]
duration = timedelta(days=5)
dt = 90

In [6]:
n = 1000   # number of particles
# Define Gaussian point cloud in the horizontal
r = 10000   # radius of particle cloud [m]
deg2m = 111000 * np.cos(50 * np.pi / 180)
var = (r / (deg2m * 3))**2
x_offset, y_offset = np.random.multivariate_normal([0, 0], [[var, 0], [0, var]], n).T
# Set a uniform distribution in depth, from dmin to dmax
dmin = 0.
dmax = 250.
zvals = dmin + np.random.random_sample(n)*(dmax-dmin)

### Simulations